In [1]:
import numpy as np
import itertools
import json, os, time
from torch.utils.data import Dataset
import torch
import copy
import pandas as pd
from absl import app
from absl import flags
import wget
from zipfile import ZipFile

In [2]:
import data_utils

param_dict: {'epochs': [200], 'batch_size': [200], 'save_every': [5], 'learning_rate': [0.001], 'test_size': [0.2], 'seed': [398], 'hidden_size': [10], 'bias': [True], 'dropout_rate': [0.1], 'ode_nn': [((50, 'tanh'), (50, 'tanh'))], 'readout_nn': [((50, 'tanh'), (50, 'tanh'))], 'enc_nn': [((50, 'tanh'), (50, 'tanh'))], 'use_rnn': [False], 'func_appl_X': [[]], 'solver': ['euler'], 'weight': [0.5], 'weight_decay': [1.0], 'dataset': ['BlackScholes', 'Heston', 'OrnsteinUhlenbeck'], 'dataset_id': [None], 'plot': [True], 'evaluate': [True], 'paths_to_plot': [(0, 1, 2, 3, 4)]}
param_dict: {'epochs': [100], 'batch_size': [20], 'save_every': [10], 'learning_rate': [0.001], 'test_size': [0.2], 'training_size': [200, 400, 800, 1600, 3200, 6400, 12800], 'seed': [398], 'hidden_size': [10], 'bias': [True], 'dropout_rate': [0.1], 'ode_nn': [((10, 'tanh'), (10, 'tanh'))], 'readout_nn': [((10, 'tanh'), (10, 'tanh'))], 'enc_nn': [((10, 'tanh'), (10, 'tanh'))], 'use_rnn': [False], 'func_appl_X': [[]], 's

In [3]:
dataset_dic = {
    'volatility': np.array([[0.05, 0.0, 0.0], [0.0, 0.1, 0.0], [0.0,0.0, 0.2]]).tolist(),
    'mean': np.array([1.2, 1.0, 1.5]).tolist(),
    'speed': np.array([[2, 0.0, 0.0], [0.0, 0.4, 0.0],[0.0,0.0,0.5]]).tolist(),
    'nb_paths': 100,
    'nb_steps': 100,
    'S0': np.array([1.0, 1.5 ,2.0]).tolist(),
    'maturity': 1.0,
    'dimension': 3,
    'obs_perc': 1.0,
    'scheme': 'euler',
    'return_vol': False,
    'v0': 1,
    'hurst': 0.75,
    'FBMmethod': 'daviesharte',
    'model_name': 'OrnsteinUhlenbeckForZ',
    'dt': 0.01,
}

In [4]:
datasetpath, dataset_id = data_utils.create_dataset("OrnsteinUhlenbeckForZ",dataset_dic)

TypeError: expected Tensor as element 0 in argument 1, but got numpy.ndarray

In [28]:
dataset_id

37

In [4]:
dataset_metadata = data_utils.load_metadata(stock_model_name="OrnsteinUhlenbeckWithCrossTerms", time_id=30)

In [6]:
dataset_metadata

{'FBMmethod': 'daviesharte',
 'S0': [1.0, 1.5, 2.0],
 'correlation': 0.5,
 'dimension': 3,
 'drift': [0.5, 0.5, 0.5],
 'dt': 0.01,
 'hurst': 0.75,
 'maturity': 1.0,
 'mean': [0.5, 0.5, 0.5],
 'model_name': 'OrnsteinUhlenbeckWithCrossTerms',
 'nb_paths': 300,
 'nb_steps': 100,
 'obs_perc': 1.0,
 'poisson_lambda': 3.0,
 'return_vol': False,
 'scheme': 'euler',
 'speed': [[0.3, 0.0, 0.0], [0.0, 0.4, 0.0], [0.0, 0.0, 0.5]],
 'v0': 1,
 'volatility': [[0.2, 0.1, 0.1], [0.1, 0.2, 0.1], [0.1, 0.1, 0.2]]}

In [5]:
from sklearn.model_selection import train_test_split

In [ ]:
train_idx, val_idx = train_test_split(np.arange(dataset_metadata["nb_paths"]), test_size=0.2)

In [ ]:
train_idx

In [29]:
data = data_utils.IrregularDataset(model_name="OrnsteinUhlenbeckWithCrossTerms", time_id=37)

In [30]:
data

In [31]:
from torch.utils.data import DataLoader

In [32]:
collate_fn, mult = data_utils.CustomCollateFnGen(None)

In [33]:
 dl = DataLoader(  # class to iterate over training data
    dataset=data, collate_fn=collate_fn,
    shuffle=True, batch_size=100, num_workers=0)

In [34]:
 for i, b in enumerate(dl):  # iterate over the dataloader
    times = b["times"]  # Produce instance of byte type instead of str type
    time_ptr = b["time_ptr"]  # pointer
    X = b["X"]
    M = b["M"]

    start_X = b["start_X"]
    obs_idx = b["obs_idx"]
    n_obs_ot = b["n_obs_ot"]
    print(i,  time_ptr.shape)

0 (101,)


In [35]:
times

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [36]:
for i, obs_time in enumerate(times): 
    
    start=time_ptr[i]
    end=time_ptr[i+1]
    X_obs = X[start:end]
    print(i, obs_time, start, end, X_obs, )

0 0.01 0 1 tensor([[1.0088, 1.5000, 2.0121, 1.0177, 2.2500, 4.0484, 1.5132, 2.0298, 3.0181]])
1 0.02 1 2 tensor([[1.0191, 1.5147, 1.9850, 1.0387, 2.2942, 3.9401, 1.5437, 2.0230, 3.0066]])
2 0.03 2 3 tensor([[1.0220, 1.5091, 1.9755, 1.0444, 2.2774, 3.9025, 1.5423, 2.0189, 2.9812]])
3 0.04 3 4 tensor([[1.0218, 1.5065, 1.9972, 1.0441, 2.2696, 3.9888, 1.5394, 2.0408, 3.0088]])
4 0.05 4 5 tensor([[1.0235, 1.5037, 1.9986, 1.0475, 2.2611, 3.9943, 1.5390, 2.0455, 3.0053]])
5 0.060000000000000005 5 6 tensor([[1.0228, 1.5146, 1.9870, 1.0461, 2.2941, 3.9481, 1.5491, 2.0322, 3.0095]])
6 0.07 6 7 tensor([[1.0221, 1.5020, 1.9285, 1.0446, 2.2561, 3.7191, 1.5352, 1.9710, 2.8966]])
7 0.08 7 8 tensor([[1.0231, 1.5067, 1.9065, 1.0468, 2.2700, 3.6348, 1.5415, 1.9506, 2.8725]])
8 0.09 8 9 tensor([[1.0322, 1.4881, 1.9004, 1.0654, 2.2144, 3.6115, 1.5360, 1.9615, 2.8279]])
9 0.09999999999999999 9 10 tensor([[1.0280, 1.4995, 1.9228, 1.0568, 2.2484, 3.6970, 1.5415, 1.9766, 2.8831]])
10 0.10999999999999999 10 11

In [40]:
times

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [41]:
times.shape

(100,)

In [42]:
time_ptr

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [43]:
X

tensor([[1.0088, 1.5000, 2.0121, 1.0177, 2.2500, 4.0484, 1.5132, 2.0298, 3.0181],
        [1.0191, 1.5147, 1.9850, 1.0387, 2.2942, 3.9401, 1.5437, 2.0230, 3.0066],
        [1.0220, 1.5091, 1.9755, 1.0444, 2.2774, 3.9025, 1.5423, 2.0189, 2.9812],
        [1.0218, 1.5065, 1.9972, 1.0441, 2.2696, 3.9888, 1.5394, 2.0408, 3.0088],
        [1.0235, 1.5037, 1.9986, 1.0475, 2.2611, 3.9943, 1.5390, 2.0455, 3.0053],
        [1.0228, 1.5146, 1.9870, 1.0461, 2.2941, 3.9481, 1.5491, 2.0322, 3.0095],
        [1.0221, 1.5020, 1.9285, 1.0446, 2.2561, 3.7191, 1.5352, 1.9710, 2.8966],
        [1.0231, 1.5067, 1.9065, 1.0468, 2.2700, 3.6348, 1.5415, 1.9506, 2.8725],
        [1.0322, 1.4881, 1.9004, 1.0654, 2.2144, 3.6115, 1.5360, 1.9615, 2.8279],
        [1.0280, 1.4995, 1.9228, 1.0568, 2.2484, 3.6970, 1.5415, 1.9766, 2.8831],
        [1.0260, 1.4993, 1.8979, 1.0526, 2.2478, 3.6020, 1.5382, 1.9472, 2.8454],
        [1.0135, 1.4918, 1.8940, 1.0271, 2.2254, 3.5874, 1.5119, 1.9196, 2.8255],
        [1.0183,

In [49]:
X.shape

torch.Size([100, 9])

In [44]:
X.shape

torch.Size([100, 9])

In [47]:
times

array([0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, 0.1 , 0.11,
       0.12, 0.13, 0.14, 0.15, 0.16, 0.17, 0.18, 0.19, 0.2 , 0.21, 0.22,
       0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3 , 0.31, 0.32, 0.33,
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.66,
       0.67, 0.68, 0.69, 0.7 , 0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77,
       0.78, 0.79, 0.8 , 0.81, 0.82, 0.83, 0.84, 0.85, 0.86, 0.87, 0.88,
       0.89, 0.9 , 0.91, 0.92, 0.93, 0.94, 0.95, 0.96, 0.97, 0.98, 0.99,
       1.  ])

In [46]:
time_ptr

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100])

In [39]:
obs_idx.shape

torch.Size([100])

In [37]:
n_obs_ot

tensor([100], dtype=torch.int32)

In [45]:
n_obs_ot.shape

torch.Size([1])

In [ ]:
 for i, obs_time in enumerate(times):
    # Propagation of the ODE until next observation
    while current_time < (obs_time - 1e-10 * delta_t):  # 0.0001 delta_t used for numerical consistency.
        if current_time < obs_time - delta_t:
            delta_t_ = delta_t
        else:
            delta_t_ = obs_time - current_time
        if self.solver == 'euler':
            h, current_time = self.ode_step(
                h, delta_t_, current_time,
                last_X=last_X[:, self.input_coords], tau=tau,
                signature=c_sig, current_y=self.readout_map(h))
            current_time_nb = int(round(current_time / delta_t))
        else:
            raise NotImplementedError

        # Storing the predictions.
        if :
            path_t.append(current_time)
            path_h.append(h)
            path_y.append(self.readout_map(h))

    # Reached an observation - only update those elements of the batch,
    #    for which an observation is made
    start = time_ptr[i]
    end = time_ptr[i + 1]
    X_obs = X[start:end]
    i_obs = obs_idx[start:end]

    if self.full_observation:
        X_obs_impute = X_obs
        use_as_input = True
    else:
        if self.masked:
            if isinstance(M, np.ndarray):
                M_obs = torch.from_numpy(M[start:end]).to(self.device)
            else:
                M_obs = M[start:end]
            M_obs_in = M_obs[:, self.input_coords]
            M_obs_out = M_obs[:, out_coords]
            M_obs_sig = M_obs[:, self.signature_coords]
        else:
            M_obs = None
            M_obs_in = None
            M_obs_out = None
            M_obs_sig = None

    # decide whether to use observation as input
    if self.training:  # check whether model is in training or eval mode
        use_as_input = self.use_observation_as_input(self.epoch)
    else:
        use_as_input = self.val_use_observation_as_input(self.epoch)

    # update signature
    if self.input_sig or self.use_sig_for_classifier:
        for ij, j in enumerate(i_obs):
            # the signature is updated only if one of the sig-coords is
            #   observed -> hence, it can happen that even though the
            #   j-th batch sample is observed, the signature is not
            #   updated because none of the sig-coords is observed
            if M_obs_sig is None or M_obs_sig[ij].sum() > 0:
                current_sig[j, :] = signature[j][current_sig_nb[j]]
                current_sig_nb[j] += 1
        if use_as_input:
            # TODO: this is not fully correct, since if we didn't
            #   use some intermediate observations, the signature still
            #   has their information when using the signature up to
            #   some later observation. However, this just means that
            #   during training, the model conditions on a (slightly)
            #   different sigma-algebra (if the signature is used), but
            #   for inference the model should still work correctly.
            #   Especially, if we are interested in predicting
            #   \hat{X}_{t,s}
            c_sig = torch.from_numpy(current_sig).float().to(
                self.device)

    # Using RNNCell to update h. Also updating loss, tau and last_X
    Y_bj = self.readout_map(h)
    if use_as_input:
        X_obs_impute = X_obs
        temp = h.clone()
        if self.masked:
            if impute:
                # self imputation only possible if input and output are
                #    the same and no quantile loss is used
                X_obs_impute = X_obs * M_obs + (torch.ones_like(
                    M_obs.long()) - M_obs) * Y_bj[i_obs.long()]
            else:
                # otherwise set all masked entries to last value of X
                X_obs_impute = X_obs * M_obs + (1-M_obs) * last_X
        c_sig_iobs = None
        if self.input_sig:
            c_sig_iobs = c_sig[i_obs]
        temp[i_obs.long()] = self.encoder_map(
            X_obs_impute[:, self.input_coords],
            mask=M_obs_in, sig=c_sig_iobs, h=h[i_obs],
            t=torch.cat((tau[i_obs], current_time - tau[i_obs]), dim=1))
        h = temp
        Y = self.readout_map(h)

        # update h and sig at last observation
        h_at_last_obs[i_obs.long()] = h[i_obs.long()].clone()
        sig_at_last_obs = c_sig
    else:
        Y = Y_bj

    if get_loss:
        # INFO: X_obs has input and output coordinates, out_coords only
        #   has the output coordinates until dim_to; Y_obs has only the
        #   output coordinates, so taking them until dim_to corresponds
        #   to the out_coords
        loss = loss + LOSS(
            X_obs=X_obs[:, out_coords], Y_obs=Y[i_obs.long(), :dim_to],
            Y_obs_bj=Y_bj[i_obs.long(), :dim_to],
            n_obs_ot=n_obs_ot[i_obs.long()], batch_size=batch_size,
            weight=self.weight, M_obs=M_obs_out)

    # make update of last_X and tau, that is not inplace
    #    (otherwise problems in autograd)
    if use_as_input:
        temp_X = last_X.clone()
        temp_tau = tau.clone()
        if self.use_y_for_ode:
            temp_X[i_obs.long()] = Y[i_obs.long()]
        else:
            temp_X[i_obs.long()] = X_obs_impute
        if self.coord_wise_tau:
            _M = torch.zeros_like(temp_tau)
            _M[i_obs] = M_obs[:, self.input_coords]
            temp_tau[_M==1] = obs_time.astype(np.float64)
        else:
            temp_tau[i_obs.long()] = obs_time.astype(np.float64)
        last_X = temp_X
        tau = temp_tau

    if return_path:
        path_t.append(obs_time)
        path_h.append(h)
        path_y.append(Y)

In [3]:
ode_nn=((50, "tanh"), (50, "tanh"))
readout_nn=((50, "tanh"), (50, "tanh"))
enc_nn=((50, "tanh"), (50, "tanh")) 

In [4]:
dataset_dic = {
    'drift': np.array([0.5, 0.5, 0.5]).tolist(),
    'volatility': np.array([[0.2, 0.1, 0.1], [0.1, 0.2, 0.1], [0.1,0.1, 0.2]]).tolist(),
    'mean': np.array([0.5, 0.5, 0.5]).tolist(),
    'poisson_lambda': 3.0,
    'speed': np.array([[0.3, 0.0, 0.0], [0.0, 0.4, 0.0],[0.0,0.0,0.5]]).tolist(),
    'correlation': 0.5,
    'nb_paths': 20000,
    'nb_steps': 100,
    'S0': np.array([1.0, 1.5 ,2.0]).tolist(),
    'maturity': 1.0,
    'dimension': 3,
    'obs_perc': 1.0,
    'scheme': 'euler',
    'return_vol': False,
    'v0': 1,
    'hurst': 0.75,
    'FBMmethod': 'daviesharte',
    'model_name': 'OrnsteinUhlenbeckMultiDimensional',
    'dt': 0.01
}

In [5]:
train.train(
    anomaly_detection=None, n_dataset_workers=None, use_gpu=None,
    nb_cpus=None, send=None, gpu_num=0,
    model_id=None, epochs=100, batch_size=100, save_every=1,
    learning_rate=0.001, test_size=0.2, seed=398,
    hidden_size=10, bias=True, dropout_rate=0.1,
    ode_nn=ode_nn, readout_nn=readout_nn,
    enc_nn=enc_nn, use_rnn=False,
    solver="euler", weight=0.5, weight_decay=1.,
    dataset="OrnsteinUhlenbeckMultiDimensional", dataset_id=23,
    plot=True, paths_to_plot=(0,),
)

{'FBMmethod': 'daviesharte', 'S0': [1.0, 1.5, 2.0], 'correlation': 0.5, 'dimension': 3, 'drift': [0.5, 0.5, 0.5], 'dt': 0.01, 'hurst': 0.75, 'maturity': 1.0, 'mean': [0.5, 0.5, 0.5], 'model_name': 'OrnsteinUhlenbeckMultiDimensional', 'nb_paths': 20000, 'nb_steps': 100, 'obs_perc': 1.0, 'poisson_lambda': 3.0, 'return_vol': False, 'scheme': 'euler', 'speed': [[0.3, 0.0, 0.0], [0.0, 0.4, 0.0], [0.0, 0.0, 0.5]], 'v0': 1, 'volatility': [[0.2, 0.1, 0.1], [0.1, 0.2, 0.1], [0.1, 0.1, 0.2]]}


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 3 is different from 9)

In [ ]:
dl = DataLoader(  # class to iterate over training data
    dataset=data_train, collate_fn=collate_fn,
    shuffle=True, batch_size=batch_size, num_workers=N_DATASET_WORKERS)